In [2]:
# import gymnasium as gym
import torch
import torch.nn as nn
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

import gymnasium as gym
from stable_baselines3.common.atari_wrappers import MaxAndSkipEnv
import ale_py
# import ale_py

torch.cuda.is_available()

True

In [3]:

class DQN(nn.Module):
    def __init__(self, nb_actions):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(4, 16, 8, stride=4),
            nn.ReLU(),
            nn.Conv2d(16, 32, 4, stride=2),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(2592, 256),
            nn.ReLU(),
            nn.Linear(256, nb_actions),
        )

    def forward(self, x):
        return self.network(x / 255.0)

In [4]:
def Deep_Q_Learning(
    env,
    device="cuda",
    nb_steps = 3_000_000,
    learning_rate=0.001
):
    q_network = DQN(env.action_space.n).to(device)
    optimizer = torch.optim.Adam(q_network.parameters(), lr=learning_rate)

    step = 0
    av_rewards=[]
    rewards = []

    progress_bar = tqdm(total=nb_steps)
    while step <= nb_steps:
        done = False
        total_rewards = 0

        obs, info = env.reset()

        while not done:
            # random policy
            action = env.action_space.sample()

            # Execute action a in emulator and observe reward rt and image xt+1
            next_obs, reward, terminated, truncated, info = env.step(action)
            progress_bar.update(1)

            done = terminated or truncated
            total_rewards += reward

            step += 1
            if (step % 5_000 == 0) and step > 0:
                av_rewards.append(np.mean(rewards))
                rewards = []
                plt.plot(av_rewards)
                plt.title("Average Reward on Breakout")
                plt.xlabel("Training Epochs")
                plt.ylabel("Average Reward per Episode")
                plt.savefig("Imgs/average_reward_on_breakout.png")
                plt.close()
                # torch.save(q_network.state_dict(), "breakout_atari_ckpt.pth")
        rewards.append(total_rewards)

In [ ]:
env = gym.make("ALE/Breakout-v5")
# env = gym.wrappers.RecordEpisodeStatistics(env)
env = gym.wrappers.ResizeObservation(env, (84, 84))
env = gym.wrappers.GrayscaleObservation(env)
env = gym.wrappers.FrameStackObservation(env, 4)
env = MaxAndSkipEnv(env, skip=4)

Deep_Q_Learning(env, device="cuda")
env.close()

  0%|          | 4980/3000000 [00:17<2:54:34, 285.92it/s]